<a href="https://colab.research.google.com/github/kimyujin0/Machine-Learning_2022/blob/main/CH5_2_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [9]:
# 교차검증(기본 5폴드 교차검증)
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, X_train, y_train)
print(scores)

{'fit_time': array([0.01990438, 0.01304221, 0.01069427, 0.01038837, 0.00973439]), 'score_time': array([0.00122929, 0.00123644, 0.00117993, 0.00131416, 0.00121784]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
# 교차검증 시 훈련 세트를 섞기
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, X_train, y_train, cv=StratifiedKFold()) #회귀모델일 경우 StratifiedKFold 사용
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
# 훈련세트를 섞은 후 10-폴드 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, X_train, y_train, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


하이퍼파라미터 튜닝

In [14]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [17]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [18]:
dt = gs.best_estimator_
print(dt.score(X_train, y_train))
# 최적의 매개변수는 best_params 속성에 저장

0.9615162593804117


In [20]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [21]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [22]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # argma() 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [23]:
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001), # 0.0001부터 0.001까지 0.0001씩 커짐
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

In [25]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [28]:
# 랜덤서치
from scipy.stats import uniform, randint

In [29]:
rgen = randint(0,10)
rgen.rvs(10)

array([3, 1, 1, 9, 3, 0, 1, 1, 8, 1])

In [30]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([122,  76, 106,  98, 108, 105,  83, 108,  90, 104]))

In [31]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.56266725, 0.76807307, 0.56182202, 0.24569576, 0.31023055,
       0.44580915, 0.17942015, 0.14714429, 0.3448856 , 0.66478913])

In [33]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25),
          }

In [34]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter=100, n_jobs=-1, random_state=42) # n_iter : 샘플링 횟수 저장
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa857598a00>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa857598490>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa857598cd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa8574ecdc0>},
                   random_state=42)

In [35]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [36]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [37]:
dt = gs.best_estimator_
print(dt.score(X_test, y_test))

0.86
